In [41]:
import pandas as pd
import glob
import os

# Dataframe zur Analyse
os.chdir = '/bundestag_klimaschutz/'
all_files = glob.glob("*.csv")

file_list = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    file_list.append(df)

df = pd.concat(file_list, axis=0, ignore_index=True)

In [42]:
df.drop('Jens Ackermann;"FDP";1975;"Sachsen-Anhalt";;"2. Juli 1975";"http://www.jens-ackermann.info";"jens.ackermann@bundestag.de"', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop(df.index[:1], inplace=True)

df = df.sort_values(by='datum')

In [43]:
import re

# Fakten zum Datensatz
anzahl_sitzungen = len(all_files)
anzahl_reden = len(df.index)

# anzahl wörter (alle Wörter)
df_new = df[df['inhalt_neu'].notnull()]

def count_words(speech):
    words = speech.split()
    word_count = len(words)
    return word_count

dfc = df_new.copy()
dfc['word_count'] = dfc.apply(lambda row: count_words(row['inhalt_neu']), axis=1)

words_total = dfc['word_count'].sum()

# Anzahl des Wortes "Klimawandel"
def count_climate(speech):
    words = speech.split()
    string = ''.join(words)
    string = str(string)
    result = re.findall("klimawandel", string)
    return len(result)

def count_any_climate_regex(speech):
    words = speech.split()
    string = ''.join(words)
    string = str(string)
    resultKlima = re.findall(r"klima\w+", string)
    resultUmwelt = re.findall(r"umwelt\w+", string)#noch genauer suchen
    return len(resultKlima) + len(resultUmwelt)

def count_klimahysterie(speech):
    words = speech.split()
    string = ''.join(words)
    string = str(string)
    result = re.findall("klimahysterie", string)
    return len(result)


dfc['Anzahl Klimawandel'] = dfc.apply(lambda row: count_climate(row['inhalt_neu']), axis=1)
dfc['Anzahl Klimahysterie'] = dfc.apply(lambda row: count_klimahysterie(row['inhalt_neu']), axis=1) 

climate_total = dfc['Anzahl Klimawandel'].sum() 
klimahysterie_total = dfc['Anzahl Klimahysterie'].sum()

def count_any_climate(speech):
    words = speech.split()
    string = ''.join(words)
    string = str(string)
    result = re.findall("klima", string) #noch genauer suchen
    return len(result)

#any_climate_total = dfc['Anzahl Klima'].sum()

In [44]:
n2_df = dfc.loc[dfc['Anzahl Klimahysterie'] != 0]

In [45]:
import numpy as np

# Filtern auf Datensätze zu Klima und Umwelt
dfc['Anzahl Klima/Umwelt'] = dfc.apply(lambda row: count_any_climate_regex(row['inhalt_neu']), axis=1) 

n3_df = dfc.loc[dfc['Anzahl Klima/Umwelt'] != 0]
n3_df = n3_df.replace(['BÜNDNIS\xa090/DIE GRÜNEN'],['BÜNDNIS 90/DIE GRÜNEN'])
n3_df = n3_df.replace(['BÜNDIS 90/DIE GRÜNEN'],['BÜNDNIS 90/DIE GRÜNEN'])
n3_df = n3_df.replace(['BÜND'],['BÜNDNIS 90/DIE GRÜNEN'])

#Filtern auf Parteien

n3_df = n3_df.loc[n3_df['fraktion'] == 'BÜNDNIS 90/DIE GRÜNEN']
print(len(n3_df))


# Fraktionen zählen
spd_counter = cdu_counter = fdp_counter = linke_counter = afd_counter = grüne_counter = fraktionslos_counter = 0 
parties_list = [spd_counter, cdu_counter, fdp_counter, linke_counter, afd_counter, grüne_counter, fraktionslos_counter]

def count_parties(party, my_list): 
    if party == "SPD":
        my_list[0] +=1
        return my_list
    elif party == "CDU/CSU":
        my_list[1] +=1
        return my_list
    elif party == "FDP":
        my_list[2] +=1
        return my_list
    elif party == "DIE LINKE":
        my_list[3] +=1
        return my_list
    elif party == "AfD":
        my_list[4] +=1
        return my_list
    elif bool(re.search("DIE GRÜNEN", str(party))) == True: 
        my_list[5] +=1
        return my_list
    elif party == 'fraktionslos' or party == 'Fraktionslos':
        my_list[6] +=1
        return my_list
    return


result = n3_df.apply(lambda row: count_parties(row['fraktion'], parties_list), axis=1)

912


In [46]:
n4_df = dfc.loc[dfc['Anzahl Klimawandel'] != 0]
len(n4_df.index)

spd_counter = cdu_counter = fdp_counter = linke_counter = afd_counter = grüne_counter = fraktionslos_counter = 0 
parties_list = [spd_counter, cdu_counter, fdp_counter, linke_counter, afd_counter, grüne_counter, fraktionslos_counter]

result2 = n4_df.apply(lambda row: count_parties(row['fraktion'], parties_list), axis=1)

In [47]:
import matplotlib.pyplot as plt
import seaborn as sns

"""
dfc['Anzahl Klimawandel rolling 7 days'] = dfc['Anzahl Klimawandel'].rolling(7, center=True).mean()
dfc.head()

plt.figure(figsize=(15,5))
plt.plot(dfc['datum'], dfc['Anzahl Klimawandel'], 'b-')
plt.plot(dfc['datum'], dfc['Anzahl Klimawandel rolling 7 days'], 'r-', label='rolling avg')
plt.xticks(rotation='60')

plt.xlabel('Datum'), plt.ylabel('Anzahl Klimawandel')
plt.show()

"""

"\ndfc['Anzahl Klimawandel rolling 7 days'] = dfc['Anzahl Klimawandel'].rolling(7, center=True).mean()\ndfc.head()\n\nplt.figure(figsize=(15,5))\nplt.plot(dfc['datum'], dfc['Anzahl Klimawandel'], 'b-')\nplt.plot(dfc['datum'], dfc['Anzahl Klimawandel rolling 7 days'], 'r-', label='rolling avg')\nplt.xticks(rotation='60')\n\nplt.xlabel('Datum'), plt.ylabel('Anzahl Klimawandel')\nplt.show()\n\n"

In [50]:
import gensim
import nltk
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# Preprocessing Topic Modeling

stop_words = stopwords.words('german')
manual_stop_words = (['werd','herr','kolleg','mehr','woll','wer', 'gut', 'wichtig','uber','konn','sag','frag', 'sag',
                      'frau', 'herr''schon', 'wurd', 'gibt', 'thema', 'ganz', 'mal', 'konn', 'glaub', 'gesagt', 'mach',
                      'geht', 'stell', 'all', 'and', 'red', 'hatt', 'debatt','mocht', 'dank', 'word', 'lieb', 'letzt',
                      'find', 'darub', 'darauf', 'desweg','eigent', 'vielleicht', 'genau', 'gar','bundesregier','deutsch', 'europa', 'mensch', 'deutschland', 'jahr', 
                     'antrag', 'sag', 'land', 'polit', 'minist', 'milliard', 'wurd', 'dam', 'uber', 'hat', 'ganz', 'prozent', 'mehr','braucht', 'frau', 'neu'])
stop_words = stop_words + manual_stop_words
stemmer = SnowballStemmer('german')
def tokenize_and_stem(text):
    
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        token = stemmer.stem(token)
        if len(token) > 2 and token not in stop_words:
            filtered_tokens.append(token)
    return filtered_tokens
long_string = n3_df.inhalt_neu.values.tolist()


In [53]:
import gensim.corpora as corpora
data_words = []
for speech in long_string:
    data_words.append(list(tokenize_and_stem(speech)))


In [54]:
#LDA Topic Modeling

id2word = corpora.Dictionary(data_words)
texts = data_words

In [55]:
corpus = [id2word.doc2bow(text) for text in texts]

In [56]:
from pprint import pprint
from gensim import models
num_topics = 10
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
#word2vec = models.Word2Vec(corpus)

In [57]:
lda_model = gensim.models.LdaMulticore(corpus=corpus_tfidf,
                                       id2word=id2word,
                                       num_topics=num_topics)

In [58]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.000*"erneuerbar" + 0.000*"kris" + 0.000*"unternehm" + 0.000*"energi" + '
  '0.000*"brauch" + 0.000*"haushalt" + 0.000*"bahn" + 0.000*"regeln" + '
  '0.000*"infrastruktur" + 0.000*"vorschlag"'),
 (1,
  '0.001*"euro" + 0.000*"haushalt" + 0.000*"ukrain" + 0.000*"gemeinsam" + '
  '0.000*"gross" + 0.000*"million" + 0.000*"erneuerbar" + 0.000*"zeit" + '
  '0.000*"-preis" + 0.000*"zukunft"'),
 (2,
  '0.000*"euro" + 0.000*"haushalt" + 0.000*"schuldenbrems" + 0.000*"afd" + '
  '0.000*"klimaschutz" + 0.000*"ukrain" + 0.000*"endlich" + 0.000*"handeln" + '
  '0.000*"digitalisier" + 0.000*"wirtschaft"'),
 (3,
  '0.000*"ministerin" + 0.000*"euro" + 0.000*"antwort" + 0.000*"gemeinsam" + '
  '0.000*"ziel" + 0.000*"gesetz" + 0.000*"regeln" + 0.000*"fdp" + '
  '0.000*"union" + 0.000*"brauch"'),
 (4,
  '0.000*"investition" + 0.000*"brauch" + 0.000*"euro" + 0.000*"wirtschaft" + '
  '0.000*"nation" + 0.000*"vereint" + 0.000*"konkret" + 0.000*"haushalt" + '
  '0.000*"kris" + 0.000*"geld"'),
 (5,
 

In [59]:
import pyLDAvis
import pickle
import pyLDAvis.gensim_models

# Visualierung der Topics

pyLDAvis.enable_notebook()

LDAvis_data_filepath =  os.path.join('./results/ldavis_prepared_'+str(num_topics))

In [60]:
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    
# deprecation warnings! 

/opt/tljh/user/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/opt/tljh/user/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/tljh/user/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/tljh/user/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/opt/tljh/user/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version

In [61]:
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.017043 -0.005972       1        1  20.526444
4     -0.003305  0.015900       2        1  18.598224
1     -0.001195  0.000780       3        1  10.710809
5      0.003368  0.003910       4        1   9.215052
6      0.002518  0.000345       5        1   8.928320
7      0.001965 -0.003016       6        1   8.349113
0      0.000106 -0.001361       7        1   7.904008
3      0.003695 -0.000734       8        1   6.202454
9      0.006192 -0.006865       9        1   6.053733
8      0.003699 -0.002987      10        1   3.511843, topic_info=               Term       Freq      Total Category  logprob  loglift
1461        bildung  33.000000  33.000000  Default  30.0000  30.0000
763   gesetzentwurf  51.000000  51.000000  Default  29.0000  29.0000
302           schul  35.000000  35.000000  Default  28.0000  28.0000
1350         regeln  30.000000  30.000000  Default  27.0000  27.0000
53              fdp  48.000000  48.000000  Default  26.0000  26.0000
...             ...        ...        ...      ...      ...      ...
339            wied   1.984374  59.367294  Topic10  -8.3201  -0.0494
601            sorg   1.913810  48.840094  Topic10  -8.3563   0.1096
1321        moglich   1.858611  45.055558  Topic10  -8.3856   0.1610
66           gesetz   1.906365  57.041283  Topic10  -8.3602  -0.0495
230       gemeinsam   1.917804  65.436821  Topic10  -8.3542  -0.1809

[764 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
9896       1  0.164501   -ausstoss
9896       2  0.082250   -ausstoss
9896       3  0.164501   -ausstoss
9896       4  0.082250   -ausstoss
9896       5  0.082250   -ausstoss
...      ...       ...         ...
14416      3  0.299893  ‑grad-ziel
14416      4  0.099964  ‑grad-ziel
14416      5  0.099964  ‑grad-ziel
14416      6  0.099964  ‑grad-ziel
14416      7  0.099964  ‑grad-ziel

[4543 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 2, 6, 7, 8, 1, 4, 10, 9])

In [44]:

from lda2vec import preprocess, Corpus

2022-06-17 16:26:54.602800: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-17 16:26:54.602863: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


ImportError: cannot import name 'preprocess' from 'lda2vec' (/opt/tljh/user/lib/python3.9/site-packages/lda2vec/__init__.py)